# **Implémentation du modèle**
---

## Introduction

Contrairement au notebook "modèle", ici nous nous intéressons aux chocs de la politique monétaire américaine sur les pays européens.



Ce notebook contient les codes nécessaire à la mise en place du modèle. Le modèle est inspiré de celui de Iacoviello et Navarro : 

- La première étape consiste à identifier les chocs de politique monétaire, en prenant le résidu de la régression du taux d'intérêt sur l'écart d'inflation par rapport à la cible et l'output gap ou le chômage. En réalité Iacoviello et Navarro utilisent des lags et valeurs présentes de l'inflation, des spreads de crédit, du PIB et des lags des taux des fonds fédéraux.
Il nous manque donc l'output gap, l'inflation et le chômage de la zone euro
- La deuxième étape consiste à estimer l'impact de ces chocs sur l'activité économique (PIB, emploi ...) en régressant ces variables sur les chocs et des variables de contrôles (4 lags du PIB et des trends linéaires et quadratiques)

In [61]:
# import des données américaines

!pip install openpyxl 
import pandas as pd

sheets_to_load = ["Daily, Close", "Monthly", "Quarterly"]  # Noms des feuilles à importer
dfs = pd.read_excel("Données_extraites/data_US.xlsx", sheet_name=sheets_to_load, header=0)

# Accéder aux DataFrames importés
dfD = dfs["Daily, Close"]
dfM = dfs["Monthly"]
dfQ = dfs["Quarterly"]

# renommer les variables
dfD.columns = dfD.iloc[0] 
dfD = dfD[1:].reset_index(drop=True)

dfM.columns = dfM.iloc[0] 
dfM = dfM[1:].reset_index(drop=True)


# Il faut trimestrialiser les données (on prend la moyenne)

dfM["date"] = pd.to_datetime(dfM["date"])  # Convertir en datetime
dfM = dfM.set_index("date")  # Définir la colonne "date" comme index
dfM_Q = dfM.resample("Q").mean()  # Appliquer le resample par trimestre

dfD["date"] = pd.to_datetime(dfD["date"])  # Convertir en datetime
dfD = dfD.set_index("date")  # Définir la colonne "date" comme index
dfD_Q = dfD.resample("Q").mean()  # Appliquer le resample par trimestre

# harmoniser les dates
dfD_Q.index = dfD_Q.index.to_period("Q")  
dfM_Q.index = dfM_Q.index.to_period("Q")

dfQ=dfQ.rename(columns={"observation_date":"date"})
dfQ = dfQ.set_index("date")
dfQ.index = dfQ.index.to_period("Q")



/tmp/ipykernel_9307/499249651.py:26: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  dfM_Q = dfM.resample("Q").mean()  # Appliquer le resample par trimestre
/tmp/ipykernel_9307/499249651.py:30: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  dfD_Q = dfD.resample("Q").mean()  # Appliquer le resample par trimestre


In [63]:
data_US = dfM_Q.merge(dfD_Q, on='date').merge(dfQ, on='date')


In [ ]:
import numpy as np
import statsmodels.api as sm

# Création d'une variable de tendance quadratique
data['TIME_PERIOD'] = pd.to_datetime(data['TIME_PERIOD'])

# Créer la variable de tendance temporelle time (1 pour le premier trimestre, 2 pour le second, etc.)
data['time'] = range(1, len(data) + 1)

# Créer la tendance quadratique time²
data['time_squared'] = data['time'] ** 2

#création des lags de time_squared

data["time_squared_lag1"] = data["time_squared"].shift(1)
data["time_squared_lag2"] = data["time_squared"].shift(2)
data["time_squared_lag3"] = data["time_squared"].shift(3)
data["time_squared_lag4"] = data["time_squared"].shift(4)